In [2260]:
'''Importing Numpy and Pandas'''
import numpy as np 
import pandas as pd 

'''Machine Learning Packages'''
from sklearn.cross_validation import cross_val_predict, cross_val_score 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,ElasticNetCV, LassoCV, RidgeCV 

'''Preprocessing Packages'''
from sklearn.preprocessing import StandardScaler, Imputer, PolynomialFeatures,Binarizer, OneHotEncoder, LabelEncoder


'''Plotting Packages'''
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline 

In [2261]:
'''Loading the CSV into the training and the testing dataframe'''
train_raw = pd.read_csv('train.csv')
test_raw = pd.read_csv('test.csv')

In [2262]:
'''Replacing the spaces in the column names with the _'''
train_raw.columns = train_raw.columns.str.lower().str.replace(' ', '_')
test_raw.columns = test_raw.columns.str.lower().str.replace(' ', '_')

In [2263]:
'''Validating the training dataset'''
train_raw.head(2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,4,2009,WD,Normal,220000


In [2264]:
'''Validating the test dataset'''
test_raw.head(2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD


In [2265]:
'''Adding a source column in the training dataset'''
train_raw['source'] = 'Train'
train_raw.head(2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice,source
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,NaN,NaN,NaN,0,3,2010,WD,Normal,130500,Train
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,NaN,NaN,NaN,0,4,2009,WD,Normal,220000,Train


In [2266]:
'''Adding a source column in the test dataset'''
test_raw['source'] = 'Test'
test_raw.head(2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,source
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Test
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,8,2006,WD,Test


In [2267]:
'''Merging the Training and the Test Dataset before creating the dummies'''
train_test_merged = train_raw.append(test_raw)
train_test_merged.head(2)

,1st_flr_sf,2nd_flr_sf,3ssn_porch,alley,bedroom_abvgr,bldg_type,bsmt_cond,bsmt_exposure,bsmt_full_bath,bsmt_half_bath,...,screen_porch,source,street,total_bsmt_sf,totrms_abvgrd,utilities,wood_deck_sf,year_built,year_remod/add,yr_sold
0,725,754,0,NaN,3,1Fam,TA,No,0.0,0.0,...,0,Train,Pave,725.0,6,AllPub,0,1976,2005,2010
1,913,1209,0,NaN,4,1Fam,TA,No,1.0,0.0,...,0,Train,Pave,913.0,8,AllPub,0,1996,1997,2009


In [2268]:
'''Validating the Shape of the Training, Test and the Merged Dataframes'''

print ('Training Shape: ', train_raw.shape)
print ('Testing Shape: ', test_raw.shape)
print ('Merge Shape: ', train_test_merged.shape)

Training Shape:  (2051, 83)
Testing Shape:  (879, 81)
Merge Shape:  (2930, 83)


In [2272]:
train_test_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2930 entries, 0 to 878
Data columns (total 83 columns):
1st_flr_sf         2930 non-null int64
2nd_flr_sf         2930 non-null int64
3ssn_porch         2930 non-null int64
alley              198 non-null object
bedroom_abvgr      2930 non-null int64
bldg_type          2930 non-null object
bsmt_cond          2850 non-null object
bsmt_exposure      2847 non-null object
bsmt_full_bath     2928 non-null float64
bsmt_half_bath     2928 non-null float64
bsmt_qual          2850 non-null object
bsmt_unf_sf        2929 non-null float64
bsmtfin_sf_1       2929 non-null float64
bsmtfin_sf_2       2929 non-null float64
bsmtfin_type_1     2850 non-null object
bsmtfin_type_2     2849 non-null object
central_air        2930 non-null object
condition_1        2930 non-null object
condition_2        2930 non-null object
electrical         2929 non-null object
enclosed_porch     2930 non-null int64
exter_cond         2930 non-null object
exter_qual     

In [2273]:
'''Retrieving the list of null columns'''
null_col_list = list(train_test_merged.columns[train_test_merged.isnull().sum()>0])
null_col_list

['alley',
 'bsmt_cond',
 'bsmt_exposure',
 'bsmt_full_bath',
 'bsmt_half_bath',
 'bsmt_qual',
 'bsmt_unf_sf',
 'bsmtfin_sf_1',
 'bsmtfin_sf_2',
 'bsmtfin_type_1',
 'bsmtfin_type_2',
 'electrical',
 'fence',
 'fireplace_qu',
 'garage_area',
 'garage_cars',
 'garage_cond',
 'garage_finish',
 'garage_qual',
 'garage_type',
 'garage_yr_blt',
 'lot_frontage',
 'mas_vnr_area',
 'mas_vnr_type',
 'misc_feature',
 'pool_qc',
 'sale_condition',
 'saleprice',
 'total_bsmt_sf']

In [2274]:
'''Length of Null Columns'''
print ('Number of Null Columns: ', len(null_col_list))

Number of Null Columns:  29


In [2275]:
'''Validating the count of the Numeric and Non-Numeric columns'''
object_col_count = 0 
numeric_col_count = 0 


for col_type in train_test_merged.dtypes: 
    if col_type =='object': 
        object_col_count += 1 
    elif col_type =='int64' or col_type =='float64': 
        numeric_col_count += 1
    else: 
        continue

print ('Object Col Count: ', object_col_count)
print ('Numberic Col Count: ', numeric_col_count)

Object Col Count:  44
Numberic Col Count:  39


In [2276]:
object_col_list = []
numeric_col_list = [] 


for k,v in enumerate(train_test_merged.dtypes):
    if v =='object': 
        object_col_list.append(train_test_merged.columns[k])
    else: 
        numeric_col_list.append(train_test_merged.columns[k])

print ('Columns of type Object:\n\n',  object_col_list)
print ('\n')
print ('Number of Object Columns: ', len(object_col_list))
print ('\n')
print ('Columns of type Int or Float:\n\n',  numeric_col_list)
print ('\n')
print ('Number of Numeric Columns: ', len(numeric_col_list))

Columns of type Object:

 ['alley', 'bldg_type', 'bsmt_cond', 'bsmt_exposure', 'bsmt_qual', 'bsmtfin_type_1', 'bsmtfin_type_2', 'central_air', 'condition_1', 'condition_2', 'electrical', 'exter_cond', 'exter_qual', 'exterior_1st', 'exterior_2nd', 'fence', 'fireplace_qu', 'foundation', 'functional', 'garage_cond', 'garage_finish', 'garage_qual', 'garage_type', 'heating', 'heating_qc', 'house_style', 'kitchen_qual', 'land_contour', 'land_slope', 'lot_config', 'lot_shape', 'mas_vnr_type', 'misc_feature', 'ms_zoning', 'neighborhood', 'paved_drive', 'pool_qc', 'roof_matl', 'roof_style', 'sale_condition', 'sale_type', 'source', 'street', 'utilities']


Number of Object Columns:  44


Columns of type Int or Float:

 ['1st_flr_sf', '2nd_flr_sf', '3ssn_porch', 'bedroom_abvgr', 'bsmt_full_bath', 'bsmt_half_bath', 'bsmt_unf_sf', 'bsmtfin_sf_1', 'bsmtfin_sf_2', 'enclosed_porch', 'fireplaces', 'full_bath', 'garage_area', 'garage_cars', 'garage_yr_blt', 'gr_liv_area', 'half_bath', 'id', 'kitchen_abv

In [2277]:
'''Reviewing the null columns types'''
train_test_merged[null_col_list].dtypes

alley              object
bsmt_cond          object
bsmt_exposure      object
bsmt_full_bath    float64
bsmt_half_bath    float64
bsmt_qual          object
bsmt_unf_sf       float64
bsmtfin_sf_1      float64
bsmtfin_sf_2      float64
bsmtfin_type_1     object
bsmtfin_type_2     object
electrical         object
fence              object
fireplace_qu       object
garage_area       float64
garage_cars       float64
garage_cond        object
garage_finish      object
garage_qual        object
garage_type        object
garage_yr_blt     float64
lot_frontage      float64
mas_vnr_area      float64
mas_vnr_type       object
misc_feature       object
pool_qc            object
sale_condition     object
saleprice         float64
total_bsmt_sf     float64
dtype: object

In [2285]:
'''Function to impute the categorical columns with the Most Frequent Values'''

def categorical_imputer(col):
    print ('Null Count in ', '"',  cols ,'"',  'column Pre Transform: ' , train_test_merged[col].isnull().sum())
    train_test_merged[col].fillna(value = train_test_merged[col].value_counts().index[0], inplace=True)
    print ('Null Count in ', '"',  cols ,'"',  'column Post Transform: ' , train_test_merged[col].isnull().sum())
    print ('\n')
    

In [2286]:
'''Testing the function on the alley column'''
categorical_imputer('alley')

Null Count in  " yr_sold " column Pre Transform:  2732
Null Count in  " yr_sold " column Post Transform:  0




In [2287]:
train_test_merged.alley.value_counts()

Grvl    2852
Pave      78
Name: alley, dtype: int64

In [2288]:
'''Using the for loop on the null columns and imputing the object columns with the most frequent value'''
for cols in null_col_list: 
    if train_test_merged[cols].dtypes == 'object': 
        categorical_imputer(cols)

Null Count in  " alley " column Pre Transform:  0
Null Count in  " alley " column Post Transform:  0


Null Count in  " bsmt_cond " column Pre Transform:  80
Null Count in  " bsmt_cond " column Post Transform:  0


Null Count in  " bsmt_exposure " column Pre Transform:  83
Null Count in  " bsmt_exposure " column Post Transform:  0


Null Count in  " bsmt_qual " column Pre Transform:  80
Null Count in  " bsmt_qual " column Post Transform:  0


Null Count in  " bsmtfin_type_1 " column Pre Transform:  80
Null Count in  " bsmtfin_type_1 " column Post Transform:  0


Null Count in  " bsmtfin_type_2 " column Pre Transform:  81
Null Count in  " bsmtfin_type_2 " column Post Transform:  0


Null Count in  " electrical " column Pre Transform:  1
Null Count in  " electrical " column Post Transform:  0


Null Count in  " fence " column Pre Transform:  2358
Null Count in  " fence " column Post Transform:  0


Null Count in  " fireplace_qu " column Pre Transform:  1422
Null Count in  " fireplace_qu 

In [2289]:
'''Validating the remaning null columms'''
train_test_merged.columns[train_test_merged.isnull().sum()>0]

Index(['bsmt_full_bath', 'bsmt_half_bath', 'bsmt_unf_sf', 'bsmtfin_sf_1',
       'bsmtfin_sf_2', 'garage_area', 'garage_cars', 'garage_yr_blt',
       'lot_frontage', 'mas_vnr_area', 'saleprice', 'total_bsmt_sf'],
      dtype='object')

### Most of the columns are numeric values, so we shall impute using the the sklearn library in later steps

# So far we have managed to extract the merge the two sets of data and also managed to impute the categorical columns. The next step will be to create the dummies for the columns with the object type

In [2292]:
train_test_merge_dummies = pd.get_dummies(train_test_merged, drop_first=True)
print (train_test_merge_dummies.shape)

(2930, 265)


In [2293]:
cols_to_exclude = ['pid', 'sale_condition', 'saleprice']

In [2294]:
cols_to_include= []
for columns in list(train_test_merge_dummies.columns): 
    if columns not in cols_to_exclude: 
        cols_to_include.append(columns)
cols_to_include[0:5]

['1st_flr_sf', '2nd_flr_sf', '3ssn_porch', 'bedroom_abvgr', 'bsmt_full_bath']

In [2295]:
train_test_merge_dummies['source_Train'].value_counts()

1    2051
0     879
Name: source_Train, dtype: int64

# Cleaned up code till this point. Next step will be to extract the test and training sets to build the model 

In [2298]:
X_new_training_data = train_test_merge_dummies.loc[train_test_merge_dummies['source_Train'] == 1,cols_to_include]
y_new_training_data = train_test_merge_dummies.loc[train_test_merge_dummies['source_Train'] == 1,'saleprice']

In [2299]:
X_new_test_data = train_test_merge_dummies.loc[train_test_merge_dummies['source_Train'] == 0,cols_to_include]

In [2300]:
X_new_training_data.drop(['garage_yr_blt', 'lot_frontage', 'mas_vnr_area'], axis=1, inplace=True)

In [2301]:
X_new_test_data.drop(['garage_yr_blt', 'lot_frontage', 'mas_vnr_area'], axis=1, inplace=True)

In [2302]:
print (X_new_training_data.shape)
print (y_new_training_data.shape)
print (X_new_test_data.shape)

(2051, 260)
(2051,)
(879, 260)


In [2327]:
# def numeric_col_extractor(df):
#     for k,v in enumerate(df.dtypes):
#         if v == 'int64' or v == 'float64' :
#             numeric_col_list_dummies.append(df.columns[k])
#         else: 
#             continue
#     return numeric_col_list_dummies


In [2328]:
# num_cols = numeric_col_extractor(X_new_training_data)
# numeric_df = X_new_training_data.loc[:,num_cols]
# im = Imputer(strategy = 'median')
# transformed_train = im.fit_transform(numeric_df)

# num_test_cols = numeric_col_extractor(X_new_training_data)
# numeric_test_df = X_new_test_data.loc[:,num_test_cols]
# transformed_test = im.transform(numeric_test_df)

In [2329]:
median_imputer = Imputer(strategy='median')

In [2330]:
'''Numeric Columns in the Training set that needs to be addressed'''
X_new_training_data.columns[X_new_training_data.isnull().sum()>0]

Index([], dtype='object')

In [2331]:
numeric_col_list_dummies=[]
for k,v in enumerate(X_new_training_data.dtypes):
    if v == 'int64' or v == 'float64' : 
        numeric_col_list_dummies.append(X_new_training_data.columns[k])

    else: 
        continue
print (numeric_col_list_dummies)

['1st_flr_sf', '2nd_flr_sf', '3ssn_porch', 'bedroom_abvgr', 'bsmt_full_bath', 'bsmt_half_bath', 'bsmt_unf_sf', 'bsmtfin_sf_1', 'bsmtfin_sf_2', 'enclosed_porch', 'fireplaces', 'full_bath', 'garage_area', 'garage_cars', 'gr_liv_area', 'half_bath', 'id', 'kitchen_abvgr', 'lot_area', 'low_qual_fin_sf', 'misc_val', 'mo_sold', 'ms_subclass', 'open_porch_sf', 'overall_cond', 'overall_qual', 'pool_area', 'screen_porch', 'total_bsmt_sf', 'totrms_abvgrd', 'wood_deck_sf', 'year_built', 'year_remod/add', 'yr_sold']


In [2332]:
'''Imputing all the numeric colums in X_Train'''
for cols in numeric_col_list_dummies:
    X_new_training_data[cols] = median_imputer.fit_transform(X_new_training_data[[cols]])


In [2333]:
X_new_training_data.columns[X_new_training_data.isnull().sum()>0]

Index([], dtype='object')

In [2311]:
y_new_training_data.isnull().sum()

0

In [2312]:
lr = LinearRegression()

In [2315]:
lr.fit(X_new_training_data, y_new_training_data)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [2316]:
lr.score(X_new_training_data, y_new_training_data)

0.94220666949129606

In [2322]:
y_predicted = lr.predict(X_new_test_data)

In [2323]:
y_predicted[0:5]

array([ 128093.77841503,  155064.8375155 ,  195676.982568  ,
        105518.6446546 ,  172272.62714757])

In [2324]:
d = {'Id': X_new_test_data['id'], 'SalePrice': y_predicted}
predicted_df= pd.DataFrame(d)
predicted_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 878
Data columns (total 2 columns):
Id           879 non-null int64
SalePrice    879 non-null float64
dtypes: float64(1), int64(1)
memory usage: 20.6 KB


In [2325]:
X_new_test_data['id'].dtype

dtype('int64')

In [2326]:
predicted_df.to_csv('ac_predictions.csv')